# mlflow Reproducibility

This notebook demonstrates mlflow's capability to reproduce results from experimental runs.

By providing the unique identifier for a run, this notebook will retrieve the model and training and test data sets for that run, retrain the model on the training data and calculate the metric score on the test data.

In [1]:
from __future__ import print_function

In [2]:
import os
import os.path
import socket
import pandas as pd
import tempfile
import pickle
import shutil
import scipy.stats
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.linear_model import ElasticNet
import mlflow
import mlflow.sklearn
import mlflow.tracking


## Set up connection to mlflow tracking server

In [3]:
host_name = "mlflow_server" #socket.gethostname()
BASE_URL = 'http://' + host_name +':5000'


client = mlflow.tracking.MlflowClient(BASE_URL)


In [4]:
for e in client.list_experiments():
    print('experiement:', e.name)

experiement: mlflow_demo1
experiement: mlflow_demo2


## Specify Run identifier to retrieve model and data

In [16]:
RUN_UUID = 'd19419a5805540ca8973e1ddd52b366c'

## Retrieve training/test data sets from run

In [17]:
# retrieve pickled data file
data_dir = client.download_artifacts(RUN_UUID,'data_sets')
print("contents of data download directory",data_dir,os.listdir(data_dir))


contents of data download directory /tmp/tmpq6pdekem/data_sets ['data.pkl']


In [18]:
os.path.join(data_dir,os.listdir(data_dir)[0])

'/tmp/tmpq6pdekem/data_sets/data.pkl'

In [19]:
# unpickle training and test data
with open(os.path.join(data_dir,os.listdir(data_dir)[0]),'rb') as f:
    data = pickle.load(f)

In [20]:
data['train_x'][:5]

array([[ 0.39624274, -0.37825504, -0.80060088, -0.9399674 , -0.02735814,
        -1.04506461,  0.54772967, -0.70723951,  0.41334757,  0.34770965,
         0.5554772 ],
       [ 0.87268093,  0.53201882,  0.62459925,  1.46345779, -0.02735814,
         0.52710899,  1.40504061,  1.68361317, -0.04858051,  1.31198399,
        -1.15740449],
       [ 0.15802364, -0.07483042, -0.12991847,  0.08148831,  0.06784456,
         0.70851364,  1.30978384, -0.0733535 ,  0.28136812, -0.87954861,
         0.71860879],
       [ 0.39624274, -0.7828212 ,  0.03775213, -0.73968196, -0.64617568,
         0.82945007,  0.30958775, -1.38246592, -0.18055996, -0.87954861,
         1.77896413],
       [-1.39040049, -0.22654273,  2.63664649, -1.04011011,  1.59108772,
        -1.10553282, -0.02381095, -0.32484088,  0.08339894, -0.6165647 ,
        -0.91270711]])

## Retrieve Model

In [21]:
# retrieve pickled data file
model_dir = client.download_artifacts(RUN_UUID,'best_estimator')
print("contents of model download directory",model_dir,os.listdir(model_dir))


contents of model download directory /tmp/tmpuwnq011u/best_estimator ['MLmodel', 'conda.yaml', 'model.pkl']


In [22]:

sklearn_model = mlflow.sklearn.load_model(model_dir)
sklearn_model

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.058512491882074746,
       max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
       n_estimators=453, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=13, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

## Train model on training data set

In [23]:
sklearn_model.fit(data['train_x'],data['train_y'].quality)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.058512491882074746,
       max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
       n_estimators=453, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=13, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

## Caclulate score on test data set

In [24]:
sklearn_model.score(data['test_x'],data['test_y'].quality)

0.5009089293046418

## Retrieve saved metric from training run

In [25]:
run_info = client.get_run(RUN_UUID)
for m in run_info.data.metrics:
    print(m.key, m.value)

r2 0.500908929305


## Clean up

In [15]:
shutil.rmtree(data_dir)
shutil.rmtree(model_dir)